In [1]:
!pip install langchain langchain-community langchain-text-splitters pymupdf faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install -U bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 139.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


In [3]:
import glob
import concurrent.futures
import os
from tqdm import tqdm
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
PDF_FOLDER = "./pdfs"
pdf_files = glob.glob(f"{PDF_FOLDER}/*.pdf")
MODEL_NAME = "intfloat/multilingual-e5-small"
INDEX_OUTPUT = "zno_faiss_index"

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [21]:
def process_single_pdf(pdf_path):
    try:
        loader = PyMuPDFLoader(pdf_path)
        return loader.load_and_split(text_splitter)
    except Exception as e:
        print(f"Error loading {pdf_path}: {e}")
        return []

In [23]:
print(f"Processing {len(pdf_files)} PDFs with {os.cpu_count()} CPU cores...")
documents = []

with concurrent.futures.ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    results = list(tqdm(executor.map(process_single_pdf, pdf_files), total=len(pdf_files)))

for res in results:
    documents.extend(res)

print(f"✅ Total chunks created: {len(documents)}")

Processing 93 PDFs with 12 CPU cores...


100%|██████████| 93/93 [00:14<00:00,  6.39it/s]

✅ Total chunks created: 49466


In [24]:
print(f"Loading embedding model: {MODEL_NAME}...")
embeddings = HuggingFaceEmbeddings(
    model_name=MODEL_NAME,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

Loading embedding model: intfloat/multilingual-e5-small...


/tmp/ipython-input-1583600352.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [27]:
print("Building FAISS index (this may take a few minutes)...")
vectorstore = FAISS.from_documents(documents, embeddings)

Building FAISS index (this may take a few minutes)...


In [29]:
vectorstore.save_local(INDEX_OUTPUT)
print(f"Index saved to {INDEX_OUTPUT}. Download this folder!")

Index saved to zno_faiss_index. Download this folder!


In [5]:
MODEL_ID = "Qwen/Qwen2.5-32B-Instruct"
INDEX_PATH = "zno_faiss_index"
EMBEDDING_MODEL = "intfloat/multilingual-e5-small"
DATA_FILE = "zno.train.jsonl"

In [9]:
print("Loading FAISS Index...")
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)
vectorstore = FAISS.load_local(
    INDEX_PATH,
    embeddings,
    allow_dangerous_deserialization=True
)
print("Index Loaded.")

Loading FAISS Index...
Index Loaded.


In [10]:
print(f"Loading {MODEL_ID}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True,
    trust_remote_code=True
)
print("Model Loaded.")

Loading Qwen/Qwen2.5-32B-Instruct...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00016-of-00017.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00017-of-00017.safetensors:   0%|          | 0.00/3.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Model Loaded.


In [11]:
def solve_question(item):
    question = item['question']

    docs = vectorstore.similarity_search(f"query: {question}", k=3)
    context = "\n".join([f"[Doc {i+1}]: {d.page_content}" for i, d in enumerate(docs)])

    options = "\n".join([f"{ans['marker']}) {ans['text']}" for ans in item['answers']])

    system_msg = "Ви — екзаменатор. Використовуйте наданий контекст для відповіді на запитання ЗНО. Якщо контекст неактуальний, ігноруйте його. Виведіть ТІЛЬКИ букву правильної відповіді."
    user_msg = f"Контекст:\n{context}\n\nПитання:\n{question}\n\nВаріанти:\n{options}\n\nВідповідь:"

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            temperature=0.1,
            do_sample=False
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer_letter = response.split("Відповідь:")[-1].strip()
    return answer_letter, context

In [14]:
TEST_LIMIT = 50  # Check 50 questions to get a reliable estimate
correct_count = 0
total_count = 0

print(f"\n🔎 --- RUNNING ACCURACY CHECK ON {TEST_LIMIT} SAMPLES ---")

with open(DATA_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= TEST_LIMIT: break

        item = json.loads(line)

        pred_letter, ctx = solve_question(item)

        true_letter = item['correct_answers'][0] if isinstance(item['correct_answers'], list) else item['correct_answers']

        is_correct = (pred_letter.strip().upper() == true_letter.strip().upper())

        if is_correct:
            correct_count += 1

        total_count += 1

        if i < 5 or i % 5 == 0:
            status = "✅" if is_correct else f"❌ (Exp: {true_letter})"
            print(f"[ID {item.get('id', i)}] {status} | Pred: {pred_letter}")
            if not is_correct:
                print(f"   Context: {ctx}...")

accuracy = (correct_count / total_count) * 100
print("-" * 50)
print(f"Final Accuracy on {total_count} samples: {accuracy:.2f}%")
print("-" * 50)

if accuracy > 60:
    print("Result: RAG is working! Proceed to submission.")
else:
    print("Result: Accuracy is low. Check if the retrieved context actually contains the answers.")


🔎 --- RUNNING ACCURACY CHECK ON 50 SAMPLES ---
[ID 0] ❌ (Exp: В) | Pred: assistant
Г
   Context: [Doc 1]: 27
	 Створи речення до кожного малюнка (усно).
.
.
.
.
.
.
Речення
.
.
.
[Doc 2]: А  кенгуру, поні, гризлі, НЛО 
     Б  шимпанзе, Токіо, івасі, метро 
     В  ООН, УПА, Хоккайдо (острів), ДАІ 
     Г  танго, цеце, журі, тріо 
2. До жіночого роду належать усі іменники в рядку 
     А  АТС, ЧАЕС, Міссурі (річка), шимпанзе 
     Б  ЗСУ,танго, піаніно, мадам 
     В  фрау, Арагві (річка), мадам, гризлі 
     Г  ООН, івасі, путасу, пані 
3. До середнього роду належать усі іменники в рядку 
     А  колібрі, ЗУНР, ГЕС, журі 
     Б  панно, алое, лібрето, жирафа 
     В метро, танго, сопрано, депо 
     Г  фортепіано, меню, леді, шосе 
4. Правильно узгоджено прикметник з іменником у рядку 
     А  широка шосе                                 
     Б  новий трюмо 
     В  маленька шимпанзе 
     Г  приємне сопрано 
    
19. Спиши текст, уставляючи пропущені літери. Обґрунтуй  
      написа